In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from collections import OrderedDict
from contextlib import contextmanager
from functools import wraps, update_wrapper
import inspect
from inspect import Parameter, getsource, getsourcefile, getfile, ismodule, \
    ismethod, isfunction

from htools import *

In [4]:
cd_root()

Current directory: /Users/hmamin/pythonhm/htools


/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:
class IndexedOrderedDict(OrderedDict):
    
    def __init__(self, data=None):
        # Argument must be iterable.
        super().__init__(data or {})
    
    def __setitem__(self, key, val):
        if isinstance(key, int):
            raise TypeError('key must not be an integer.')
        super().__setitem__(key, val)
        
    def __getitem__(self, key):
        if isinstance(key, int):
            return list(self.values())[key]
        return super().__getitem__(key)

In [59]:
d = {'dog': 1, 'cat': 33, 'horse': -4, 'donkey': [5]}
od = OrderedDict(d)
iod = IndexedOrderedDict(d)
iod

IndexedOrderedDict([('dog', 1), ('cat', 33), ('horse', -4), ('donkey', [5])])

In [60]:
iod['fish'] = 100
with assert_raises(TypeError):
    iod[0] = 99
iod

As expected, got TypeError(key must not be an integer.).


IndexedOrderedDict([('dog', 1),
                    ('cat', 33),
                    ('horse', -4),
                    ('donkey', [5]),
                    ('fish', 100)])

In [61]:
iod['horse']

-4

In [62]:
iod['cat']

33

In [68]:
with assert_raises(IndexError):
    iod[9]

As expected, got IndexError(list index out of range).


In [64]:
iod[3]

[5]

In [65]:
iod.move_to_end('donkey')
iod

IndexedOrderedDict([('dog', 1),
                    ('cat', 33),
                    ('horse', -4),
                    ('fish', 100),
                    ('donkey', [5])])

In [66]:
iod[3]

100

In [67]:
save(iod, 'data/tmp.pkl')

Writing data to data/tmp.pkl.


In [54]:
load('data/tmp.pkl')

Object loaded from data/tmp.pkl.


IndexedOrderedDict([('dog', 1),
                    ('cat', 33),
                    ('horse', -4),
                    ('fish', 100),
                    ('donkey', [5])])

## Function Interface

In [12]:
@valuecheck
def function_interface(present=(), required=(), defaults=(), startswith=(),
                       args:(True, False, None)=None, 
                       kwargs:(True, False, None)=None,
                       like_func=None):
    def decorator(func):
        def _param_status(param, params):
            if param not in params:
                return 'missing'
            if params[param].default == inspect._empty:
                return 'required'
            return 'optional'
        
        params = inspect.signature(func).parameters
        name = func_name(func)
        for param in present:
            if param not in params:
                raise RuntimeError(
                    f'`{name}` signature must include parameter {param}.'
                )
        for param in required:
            if _param_status(param, params) != 'required':
                raise RuntimeError(
                    f'`{name}` signature must include parameter {param} with '
                    'no default parameter.'
                )
        for param in defaults:
            if _param_status(param, params) != 'optional':
                raise RuntimeError(
                    f'`{name}` signature must include parameter {param} with '
                    'default value.'
                )
        params_list = list(params.keys())
        for i, param in enumerate(startswith):
            if params_list[i] != param:
                raise RuntimeError(f'`{name}` signature\'s parameter #{i+1} '
                                   f'(1-indexed) must be named {param}.')
        if args is not None:
            has_args = any(v.kind == Parameter.VAR_POSITIONAL 
                           for v in params.values())
            if has_args != args:
                raise RuntimeError(f'`{name}` signature must '
                                   f'{"" if args else "not"} accept *args.')
        if kwargs is not None:
            has_kwargs = any(v.kind == Parameter.VAR_KEYWORD
                             for v in params.values())
            if has_kwargs != kwargs:
                raise RuntimeError(
                    f'`{name}` signature must {"" if kwargs else "not"} '
                    'accept **kwargs.'
                )
        if like_func and str(signature(like_func)) != str(signature(func)):
            raise RuntimeError(f'`{name}` signature must match {like_func} '
                               'signature.')
        @wraps(func)
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)
        return wrapper
    return decorator

In [7]:
feature_func = function_interface(['b'], ['a'], ['c', 'd', 'e'], kwargs=True)

In [8]:
@feature_func
def foo(a, b, c=4, d=4, e=5, **kwargs):
    return a, b, c, e

In [9]:
@feature_func
def foo(a, c=4, d=4, e=5):
    return a, b, c, e

RuntimeError: `foo` signature must include parameter b.

In [197]:
@feature_func
def foo(a, b, c=4, e=5):
    return a, b, c, e

RuntimeError: `foo` signature must include parameter d with default value.

In [198]:
@feature_func
def foo(a, b, d, c=4, e=5):
    return a, b, c, d, e

RuntimeError: `foo` signature must include parameter d with default value.

In [ ]:
@feature_func
def foo(a, b=0, d=1, c=4, e=5):
    return a, b, c, d, e

In [199]:
@feature_func
def foo(b, a, d=1, c=4, e=5, **kwargs):
    return a, b, c, d, e

In [204]:
fit_func = function_interface(startswith=['x', 'y', 'z'], kwargs=True)

In [205]:
@fit_func
def bar(x, y, z=3, *args, **kwargs):
    return x, y, z

In [206]:
@fit_func
def bar(x, z, y, *args, **kwargs):
    return x, y, z

RuntimeError: `bar` signature's parameter #2 (1-indexed) must be named y.

In [207]:
@fit_func
def bar(x, y=0, z=-2):
    return x, y, z

RuntimeError: `bar` signature must  accept **kwargs.